In [18]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import random

class Gomoku:
    def __init__(self, size=15, win_length=5):
        self.size = size
        self.win_length = win_length
        self.reset()

    def reset(self):
        self.board = np.zeros((self.size, self.size), dtype=int)
        self.current_player = 1
        self.move_count = 0
        return self.board

    def step(self, action):
        x, y = action
        if not (0 <= x < self.size and 0 <= y < self.size):
            print("无效动作：超出棋盘边界。")
            return self.board, -10, True  # 非法动作，返回负奖励
        if self.board[x, y] != 0:
            # 寻找一个新的空位
            while (0 <= x < self.size and 0 <= y < self.size and self.board[x, y] != 0):
                x, y = self.random_empty_position()
            print(f"原位置被占，自动跳转到新位置：{x, y}")
        self.board[x, y] = self.current_player
        self.move_count += 1
        if self.check_win(x, y):
            return self.board, 1, True
        if self.move_count >= self.size * self.size:  # 如果棋盘满了，平局
            return self.board, 0, True
        self.current_player = 3 - self.current_player
        return self.board, 0, False

    def check_win(self, x, y):
        # 检查胜利条件的逻辑
        directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
        for dx, dy in directions:
            if self.count_consecutive(x, y, dx, dy) >= self.win_length:
                return True
        return False

    def count_consecutive(self, x, y, dx, dy):
        # 计算连续相同棋子的数量
        count = 1
        i, j = x + dx, y + dy
        while 0 <= i < self.size and 0 <= j < self.size and self.board[i, j] == self.current_player:
            count += 1
            i += dx
            j += dy
        return count

    def random_empty_position(self):
        # 返回一个随机的空位
        free_positions = [(i, j) for i in range(self.size) for j in range(self.size) if self.board[i, j] == 0]
        return random.choice(free_positions)

    def available_actions(self):
        return [(i, j) for i in range(self.size) for j in range(self.size) if self.board[i, j] == 0]

def display_board(board):
    clear_output(wait=True)
    plt.figure(figsize=(8, 8))
    plt.imshow(board, cmap='coolwarm', extent=(0, board.shape[0], 0, board.shape[1]))
    plt.grid(which='both')
    plt.xticks(np.arange(board.shape[0]+1)-0.5, [''] * (board.shape[0] + 1))
    plt.yticks(np.arange(board.shape[1]+1)-0.5, [''] * (board.shape[1] + 1))
    plt.show()

# 智能体和训练逻辑的代码将基于这个环境类进行实现。

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque

class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class DQNAgent:
    def __init__(self, input_dim, output_dim, learning_rate=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.1, batch_size=64, memory_size=10000):
        self.model = DQN(input_dim, output_dim)
        self.target_model = DQN(input_dim, output_dim)
        self.target_model.load_state_dict(self.model.state_dict())
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.memory = deque(maxlen=memory_size)
        self.loss_fn = nn.MSELoss()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size

    def act(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.output_dim)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            actions_values = self.model(state)
        return torch.argmax(actions_values, dim=1).item()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        batch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in batch:
            state = torch.FloatTensor(state).unsqueeze(0)
            next_state = torch.FloatTensor(next_state).unsqueeze(0)
            target = reward + self.gamma * torch.max(self.target_model(next_state)[0]) * (1 - done)
            action_values = self.model(state)
            action_values[0][action] = target
            loss = self.loss_fn(action_values, torch.tensor([action]).unsqueeze(0))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

# 训练逻辑
def train_dqn(env, agent1, agent2, num_episodes):
    win_counts = {1: 0, 2: 0}
    for e in range(num_episodes):
        state = torch.FloatTensor(env.reset().flatten()).unsqueeze(0)
        done = False
        current_agent = agent1
        while not done:
            available_actions = [a for a in env.available_actions()]
            if not available_actions:
                print("错误：没有可用的动作。")
                break

            action = current_agent.act(state[0].numpy())
            while action not in available_actions:
                action = current_agent.act(state[0].numpy())  # 重新选择动作直到找到有效动作

            action_pos = (action // env.size, action % env.size)
            next_state, reward, done = env.step(action_pos)
            next_state = torch.FloatTensor(next_state.flatten()).unsqueeze(0)
            current_agent.remember(state[0].numpy(), action, reward, next_state[0].numpy(), done)

            if done:
                print(f"Episode {e+1}/{num_episodes}, Winner: Player {env.current_player}, Reward: {reward}, Moves: {env.move_count}")
                if reward == 1:
                    win_counts[env.current_player] += 1
                break

            state = next_state
            current_agent = agent2 if current_agent == agent1 else agent1

        agent1.replay()
        agent2.replay()

# 初始化环境和智能体
env = Gomoku()
input_dim = env.size * env.size
output_dim = env.size * env.size
agent1 = DQNAgent(input_dim, output_dim)
agent2 = DQNAgent(input_dim, output_dim)

# 训练智能体
train_dqn(env, agent1, agent2, num_episodes=1000)

KeyboardInterrupt: 

In [ ]:
def model_vs_model(env, agent1, agent2, display=True):
    state = torch.FloatTensor(env.reset().flatten()).unsqueeze(0)
    done = False
    current_agent = agent1
    turn = 1
    while not done:
        if display:
            display_board(env.board)

        available_actions = [a for a in env.available_actions()]
        if not available_actions:  # 如果没有可用动作，显示平局信息并结束游戏
            print("平局！")
            break

        action = current_agent.act(state[0].numpy())
        while action not in available_actions:
            action = current_agent.act(state[0].numpy())  # 重新选择动作直到找到有效动作

        action_pos = (action // env.size, action % env.size)
        next_state, reward, done = env.step(action_pos)
        next_state = torch.FloatTensor(next_state.flatten()).unsqueeze(0)
        print(f"模型{turn}落子位置：{action_pos}")

        if done:
            if reward == 1:
                print(f"模型{turn}获胜！")
            elif reward == -1:
                print(f"模型{3 - turn}获胜！")
            break

        state = next_state
        current_agent = agent2 if turn == 1 else agent1
        turn = 3 - turn  # Switch turn

# 展示两个智能体之间的对抗
model_vs_model(env, agent1, agent2)